In [75]:
import pandas
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from pandas_profiling import ProfileReport
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import roc_auc_score


In [6]:
dataframe = pandas.read_csv('./superstore_data.csv')

In [10]:
profile = ProfileReport(dataframe)
profile.to_file(output_file='report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Luego de haber realizado el analisis exploratorio se pudo observar que la columna "incoming" tiene una cantidad del 1.1% de registros vacios, se procede a eliminarlos

In [45]:
clean_dataframe = dataframe.dropna()

In [46]:
clean_dataframe.Education = clean_dataframe.Education.map({'Graduation': 1, 'PhD': 2, 'Master':3, '2n Cycle': 4, 'Basic': 5})

/tmp/ipykernel_7499/1202804512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataframe.Education = clean_dataframe.Education.map({'Graduation': 1, 'PhD': 2, 'Master':3, '2n Cycle': 4, 'Basic': 5})


In [47]:
clean_dataframe.Marital_Status = clean_dataframe.Marital_Status.map({
    'Married': 1, 
    'Together': 2, 
    'Single':3, 
    'Divorced': 4, 
    'Widow': 5,
    'Alone': 5,
    'YOLO': 5,
    'Absurd': 5
})

/tmp/ipykernel_7499/2575629651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataframe.Marital_Status = clean_dataframe.Marital_Status.map({


In [48]:
clean_dataframe['Dt_Customer'] = pandas.to_datetime(clean_dataframe['Dt_Customer'], errors="coerce")
clean_dataframe['Dt_Customer'] = clean_dataframe['Dt_Customer'].astype(int).div(10**9)

/tmp/ipykernel_7499/955561588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataframe['Dt_Customer'] = pandas.to_datetime(clean_dataframe['Dt_Customer'], errors="coerce")
/tmp/ipykernel_7499/955561588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_dataframe['Dt_Customer'] = clean_dataframe['Dt_Customer'].astype(int).div(10**9)


In [49]:
clean_dataframe.dtypes

Id                       int64
Year_Birth               int64
Education                int64
Marital_Status           int64
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer            float64
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
Response                 int64
Complain                 int64
dtype: object

In [51]:
dataframe.shape

(2240, 22)

In [52]:
clean_dataframe.shape

(2216, 22)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(
    clean_dataframe.drop(['Response'], axis=1), 
    clean_dataframe.Response, 
    train_size=0.75,
    test_size=0.25,
    random_state=42
)

In [76]:
pipeline_optimizer = TPOTClassifier(
    verbosity = 2,
    generations= 10,
    memory = 'auto',
    n_jobs = -1,
    scoring = 'f1'
)

In [77]:
def run_model(X_train, y_train, X_test, y_test, filename_model):
    pipeline_optimizer.fit(X_train, y_train)
    print(pipeline_optimizer.score(X_test, y_test))
    pipeline_optimizer.export(filename_model)

    predictions = pipeline_optimizer.predict(X_test)
    print("AUC score: ")
    print(roc_auc_score(y_test, predictions))

# Tecnicas de balanceo

### Estrategia: Subsampling en la clase mayoritaria

In [67]:
us = NearMiss(n_neighbors=3, version=2)
X_train_res, y_train_res = us.fit_resample(X_train, y_train)
run_model(X_train_res, y_train_res, X_test, y_test, 'near_miss_model.py')

### Estrategia: Oversampling de la clase minoritaria

In [78]:
overSample =  RandomOverSampler()
X_train_res, y_train_res = overSample.fit_resample(X_train, y_train)
run_model(X_train_res, y_train_res, X_test, y_test, 'over_sample_model.py')


Optimization Progress:   0%|          | 0/1100 [00:00<?, ?pipeline/s]

/home/ramses/Documentos/proyects/faa/superstore/.venv/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ramses/Documentos/proyects/faa/superstore/.venv/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ramses/Documentos/proyects/faa/superstore/.venv/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ramses/Documentos/proyects/faa/supers


Generation 1 - Current best internal CV score: 0.983959506773431

Generation 2 - Current best internal CV score: 0.983959506773431

Generation 3 - Current best internal CV score: 0.983959506773431

Generation 4 - Current best internal CV score: 0.983959506773431

Generation 5 - Current best internal CV score: 0.9846585857822969

Generation 6 - Current best internal CV score: 0.987070722561012


TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(StandardScaler(input_matrix), bootstrap=False, criterion=gini, max_features=0.1, min_samples_leaf=1, min_samples_split=3, n_estimators=100)
0.3826086956521739
AUC score: 
0.6204283442655536


# Clasificador

In [68]:
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/5100 [00:00<?, ?pipeline/s]

/home/ramses/Documentos/proyects/faa/superstore/.venv/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ramses/Documentos/proyects/faa/superstore/.venv/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ramses/Documentos/proyects/faa/superstore/.venv/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ramses/Documentos/proyects/faa/supers


Generation 1 - Current best internal CV score: 0.49241295139524865

Generation 2 - Current best internal CV score: 0.49241295139524865

Generation 3 - Current best internal CV score: 0.49241295139524865

Generation 4 - Current best internal CV score: 0.4953661147808358

Generation 5 - Current best internal CV score: 0.4953661147808358

Generation 6 - Current best internal CV score: 0.4976039938651283

Generation 7 - Current best internal CV score: 0.522324135646768

Generation 8 - Current best internal CV score: 0.522324135646768

Generation 9 - Current best internal CV score: 0.522324135646768

Generation 10 - Current best internal CV score: 0.522324135646768

Generation 11 - Current best internal CV score: 0.522324135646768

Generation 12 - Current best internal CV score: 0.522324135646768

Generation 13 - Current best internal CV score: 0.5244906479211346

Generation 14 - Current best internal CV score: 0.5284174317958233

Generation 15 - Current best internal CV score: 0.528417431

TPOTClassifier(generations=50, memory='auto', n_jobs=-1, scoring='f1',
               verbosity=2)

In [69]:
print(pipeline_optimizer.score(X_test, y_test))

0.5222222222222221
